In [1]:
import pandas as pd
from datasets import load_dataset, combine

import sys
sys.path.append("/home/st-gorbatovski/sollama")

from src.mpnet_reward.data.make_dataset import make_datasets
from src.mpnet_reward.utils import set_random_seed

/home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/st-gorbatovski/.conda/envs/gorbatovski_env did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/var/lib/nvidia-mig-manager/checkpoint.json')}
  warn(msg)
/home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/st-gorbatovski/.conda/envs/gorbatovski_env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:

In [2]:
set_random_seed(42)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-mpnet-base-cos-v1")

In [4]:
data_config = {
    "dataset_name": "Myashka/SO_Python_basics_QA_human_pref",
    "max_length_q": 512,
    "max_length_a": 256,
    "use_title": True,
    "val_size": 500,
    "test_size": 1000,
    "use_gen": False,
    "use_par": True,
    "use_so": True,
    "num_proc": 8,
    "so_fix": 0.15
}

In [5]:
test_dataset = make_datasets(tokenizer=tokenizer, **data_config, do_train=False)

Found cached dataset json (/home/st-gorbatovski/.cache/huggingface/datasets/Myashka___json/Myashka--SO_Python_basics_QA_human_pref-0870bdfdc441eea5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<00:00, 14.41it/s]
Loading cached processed dataset at /home/st-gorbatovski/.cache/huggingface/datasets/Myashka___json/Myashka--SO_Python_basics_QA_human_pref-0870bdfdc441eea5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-0a0547a904deef09_*_of_00008.arrow
Loading cached processed dataset at /home/st-gorbatovski/.cache/huggingface/datasets/Myashka___json/Myashka--SO_Python_basics_QA_human_pref-0870bdfdc441eea5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-c739cf9d2508d519_*_of_00008.arrow
Loading cached processed dataset at /home/st-gorbatovski/.cache/huggingface/datasets/Myashka___json/Myashka--SO_Python_basics_QA_human_pref-0870bdfdc441eea5/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd

Statistics for Test Dataset:
Total: 1000
Generated (k): 0 (0.00%)
Paraphrased (k only): 343 (34.30%)
Paraphrased (j only): 0 (0.00%)
Paraphrased (both j & k): 507 (50.70%)
SO (k): 150 (15.00%)
SO (j): 493 (49.30%)
Total Paraphrased: 850 (85.00%)
Total SO: 150 (15.00%)
--------------------------------------------------


## Создание набора данных для генерации набора и валидации теста Reward model

In [9]:
df = test_dataset.to_pandas()

In [10]:
dataset = load_dataset("Myashka/SO-Python_basics_QA-filtered-2023-tanh_score")

Found cached dataset json (/home/st-gorbatovski/.cache/huggingface/datasets/Myashka___json/Myashka--SO-Python_basics_QA-filtered-2023-tanh_score-65cea773c91b5e47/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 3/3 [00:00<00:00, 191.05it/s]


In [11]:
dataset = combine.concatenate_datasets([dataset['train'],dataset['validation'], dataset['test']]).filter(lambda x: x["Q_Id"] in df['Q_Id'].unique())

In [13]:
to_gen_df = dataset.to_pandas().sort_values('log_score', ascending=False).drop_duplicates(subset=["Q_Id"], keep='first').reset_index(drop=True)

In [14]:
to_gen_df = to_gen_df.drop(columns=["Available Count"])

In [ ]:
to_gen_df.to_json("/home/st-gorbatovski/sollama/data/processed/tanh_score/python_basics/unique_test_to_gen_reward_eval_v2.json", orient="records")

In [ ]:
pd.read_json("/home/st-gorbatovski/sollama/data/processed/tanh_score/python_basics/unique_test_to_gen_reward_eval.json")